# PHASTEST_API_to_VIZ

노트북 **세 번째 셀**의 Working directory 설정과 **네 번째 셀**의 변수 지정을 한 다음,  
처음부터 셀을 쭉 실행해주시면 PHASTEST 결과 및 이를 시각화할 수 있는 JSON 파일이 생성됩니다.

### Output 파일 구조
<pre>
── {SAMPLE_NAME}_results/  
   ├── {SAMPLE_NAME}/  ← PHASTEST 결과 디렉토리  
   │   ├── ...  
   │   └── ...  
   └── {SAMPLE_NAME}_output_for_viz.json   ← 시각화용 JSON 파일  
</pre>

### 시각화 방법
`Phage_gene_visualizer.html` 파일을 더블클릭해서 연 뒤 생성된 시각화용 JSON 파일(`{SAMPLE_NAME}_output_for_viz.json`)을 업로드하면 시각화가 가능합니다.

In [6]:
import requests
import json
import base64
import os
import zipfile
import io
import time

In [ ]:
# RunPod Serverless Endpoint 정보

API_KEY = "rpa_KDY2Q6WE4GGJZYDQONYNHOVKY3FAVJM84F66NZJLykvysx"
ENDPOINT_ID = "hmhrwi5mvm8idm"

In [ ]:
# Working directory 설정

os.chdir("/home/es/workspace/phastest_docker_build/test")

In [ ]:
# 변수 지정

INPUT_TYPE = "fasta"                  # option : {fasta|contig|genbank}
MODE = "lite"                         # option : {lite|deep}
SAMPLE_NAME = "CD_7908"               # 샘플 이름 (ex. CD_7908)
FASTA_FILE = "GCF_051630995.1.fasta"  # FASTA 파일 경로 (ex. ./GCF_051630995.1.fasta)
ACCESSION = None                      # NCBI Genbank Accession # (ex. KF030445.1), fasta 또는 contig 모드일 경우 None

In [ ]:
# RunPod input 생성

if INPUT_TYPE in ("fasta", "contig"):
    if not os.path.exists(FASTA_FILE):
        raise FileNotFoundError(f"입력 파일을 찾을 수 없습니다: {FASTA_FILE}")

    with open(FASTA_FILE, "rb") as f:
        fasta_b64 = base64.b64encode(f.read()).decode("utf-8")

    payload = {
        "input": {
            "input_type": INPUT_TYPE,
            "mode": MODE,
            "sample_name": SAMPLE_NAME,
            "fasta_file": fasta_b64
        }
    }
elif INPUT_TYPE == "genbank":
    if not ACCESSION:
        raise ValueError("genbank 입력 시 ACCESSION을 지정해야 합니다.")
    payload = {
        "input": {
            "input_type": "genbank",
            "mode": MODE,
            "sample_name": SAMPLE_NAME,
            "accession": ACCESSION
        }
    }
else:
    raise ValueError(f"지원하지 않는 input_type입니다: {INPUT_TYPE}")

print("RunPod input JSON:")
if INPUT_TYPE in ("fasta", "contig"):
    print(json.dumps({
        "input": {
            "input_type": payload["input"]["input_type"],
            "mode": payload["input"]["mode"],
            "sample_name": payload["input"]["sample_name"],
            "fasta_file": f"<{len(payload['input']['fasta_file'])} base64 chars>"
        }
    }, indent=2))
elif INPUT_TYPE == "genbank":
    print(json.dumps({
        "input": {
            "input_type": payload["input"]["input_type"],
            "mode": payload["input"]["mode"],
            "sample_name": payload["input"]["sample_name"],
            "accession": payload["input"]["accession"]
        }
    }, indent=2))

RunPod input JSON:
{
  "input": {
    "input_type": "fasta",
    "mode": "lite",
    "sample_name": "CD_7908",
    "fasta_file": "<5659832 base64 chars>"
  }
}


In [5]:
# API 호출

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

url = f"https://api.runpod.ai/v2/{ENDPOINT_ID}/run"
response = requests.post(url, headers=headers, json=payload, verify = False)


resp_json = response.json()

if "id" not in resp_json:
    raise RuntimeError(f"Job 생성 실패: {resp_json}")

job_id = resp_json["id"]
print(f"Job ID: {job_id}")

/home/es/miniforge3/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.runpod.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Job ID: 9aedc229-3bc0-4961-9f0d-707bf46400ef-e1


In [ ]:
# JOB 상태 확인

url_status = f"https://api.runpod.ai/v2/{ENDPOINT_ID}/status/{job_id}"
'''
# 수동 확인
status_resp = requests.get(url_status, headers=headers, verify = False).json()
status = status_resp.get("status")

if status in ["COMPLETED", "FAILED"]:
    print(f"\nStatus: {status}")
else :
    print("\nStatus : Working...")
'''


# 자동 확인
while True:
    status_resp = requests.get(url_status, headers=headers, verify = False).json()
    status = status_resp.get("status")

    if status in ["COMPLETED", "FAILED"]:
        print(f"\nJob 상태: {status}")
        break

    print("...Job 실행 중 (60초 후 다시 확인)")
    time.sleep(60)



/home/es/miniforge3/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.runpod.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...Job 실행 중 (60초 후 다시 확인)


/home/es/miniforge3/envs/jupyter/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.runpod.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Job 상태: COMPLETED


In [ ]:
# 결과 JSON 저장

result = status_resp["output"]
output_json = f"result_{SAMPLE_NAME}.json"
with open(output_json, "w") as f:
    json.dump(result, f, indent=2)

print(f"\nRunPod 실행 결과 (JSON) : {output_json}")


RunPod 실행 결과 : result_vB_PaeM_E217.json


In [8]:
# JSON에서 ZIP 결과 추출 및 저장

if "result_zip_b64" not in result:
    print("\nZIP 결과가 포함되어 있지 않습니다. stderr를 확인하세요.")
    print("stderr :", result.get("stderr", "stderr 비어 있음"))
else:
    zip_data = base64.b64decode(result["result_zip_b64"])
    zip_filename = f"{SAMPLE_NAME}_results.zip"

    # ZIP 파일로 저장
    with open(zip_filename, "wb") as f:
        f.write(zip_data)

    print(f"\n결과 ZIP 파일 저장 완료 → {zip_filename}")

    # 압축 해제
    extract_dir = f"{SAMPLE_NAME}_results"
    os.makedirs(extract_dir, exist_ok=True)

    with zipfile.ZipFile(io.BytesIO(zip_data), "r") as zf:
        zf.extractall(extract_dir)

    print(f"결과 압축 해제 완료 → 폴더: {extract_dir}")

    # 내부 파일 목록 출력
    print("\n포함된 파일:")
    for root, _, files in os.walk(extract_dir):
        for file in files:
            print(" -", os.path.join(root, file))


결과 ZIP 파일 저장 완료 → vB_PaeM_E217_results.zip
결과 압축 해제 완료 → 폴더: vB_PaeM_E217_results

포함된 파일:
 - vB_PaeM_E217_results/NC_042079/json_input
 - vB_PaeM_E217_results/NC_042079/NC_042079.fna
 - vB_PaeM_E217_results/NC_042079/region_DNA.txt
 - vB_PaeM_E217_results/NC_042079/timing_messages
 - vB_PaeM_E217_results/NC_042079/summary.txt
 - vB_PaeM_E217_results/NC_042079/success.txt
 - vB_PaeM_E217_results/NC_042079/NC_042079.done
 - vB_PaeM_E217_results/NC_042079/json_input_regions
 - vB_PaeM_E217_results/NC_042079/NC_042079.log
 - vB_PaeM_E217_results/NC_042079/detail.txt


In [ ]:
# 이후부터는 Visualize용 파일 변환에 관련된 코드입니다.
import json
import re
import random

In [ ]:
# 필요 함수 정의

# 색상 생성 (RGB)
def random_color():
    return f"rgb({random.randint(50, 200)}, {random.randint(50, 200)}, {random.randint(50, 200)})"

# 로그 파일에서 sequence length 추출
def parse_seq_length(log_path):
    with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            match = re.search(r"seq length\s*=\s*(\d+)", line)
            if match:
                return int(match.group(1))
    raise ValueError("로그 파일에서 'seq length = NNN' 패턴을 찾을 수 없습니다.")

# PHASTEST JSON → CGView JSON 변환
def convert_to_cgview():
    phastest_json_path = f"{SAMPLE_NAME}_results/{SAMPLE_NAME}/json_input"
    log_path = f"{SAMPLE_NAME}_results/{SAMPLE_NAME}/{SAMPLE_NAME}.log"
    output_path = f"{SAMPLE_NAME}_results/{SAMPLE_NAME}_output_for_viz.json"

    # 1. JSON 로드
    with open(phastest_json_path, "r", encoding="utf-8") as f:
        features_raw = json.load(f)

    # 2. sequence 길이 추출
    seq_length = parse_seq_length(log_path)

    # 3. features 변환
    features = []
    legends = set()

    for item in features_raw:
        start = int(item.get("start", 0))
        stop = int(item.get("stop", 0))
        if stop <= 0:
            continue

        # 매핑 규칙 적용
        type_val = item.get("type", "Unknown")
        name_val = item.get("name", "Unknown")
        strand_val = 1 if item.get("strand", "+") == "+" else -1
        source_val = item.get("phage_bac_class", "Unknown").capitalize()

        region_index_val = item.get("region_index", "?")
        if isinstance(region_index_val, (int, float)) or str(region_index_val).isdigit():
            legend_val = f"Phage {region_index_val}"
        else:
            legend_val = str(region_index_val)
        legends.add(legend_val)

        features.append({
            "type": type_val,
            "name": name_val,
            "start": start,
            "stop": stop,
            "strand": strand_val,
            "source": source_val,
            "legend": legend_val
        })

    # 4. legend items 생성
    legend_items = []
    for region in sorted(legends):
        legend_items.append({
            "name": region,
            "swatchColor": random_color(),
            "decoration": "arrow"
        })

    # 5. CGView 포맷 조립
    cgview_json = {
        "cgview": {
            "version": "1.7.0",
            "sequence": {"length": seq_length},
            "features": features,
            "legend": {"items": legend_items},
            "tracks": [
                {
                    "name": "Phage",
                    "dataType": "feature",
                    "dataMethod": "source",
                    "dataKeys": "Phage"
                }
            ]
        }
    }

    # 6. JSON 파일 저장
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(cgview_json, f, indent=2)

    print(f"변환 완료! : {output_path}")

In [27]:
# 실행
convert_to_cgview()

변환 완료! : CD_7908_results/CD_7908_output_for_viz.json
